In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
import pandas as pd
import keras
from keras.layers import Dense, LSTM, Dropout, Conv2D, Lambda, MaxPooling2D, Flatten
from sklearn.preprocessing import MinMaxScaler
from tqdm.notebook import tqdm
from EEGPipelineModule import read_gdf, create_epochs, downsampling_epochs, epochs_to_dataframe, filtering_epochs

#### Functions for dataframe preparation

In [ ]:
def df_to_x_y(df):
    x = df.set_index('time')
    x.drop('epoch', axis=1, inplace=True)
    y = pd.get_dummies(x.condition)
    x.drop('condition', axis=1, inplace=True)
    return x, y


def x_to_3d(x):
    scaler = MinMaxScaler()
    x_scaled = pd.DataFrame(scaler.fit_transform(x), columns=x.columns, index=x.index)
    x_3d = x_scaled.values.reshape((n_samples,-1,n_channels))
    return x_3d



def y_to_3d(y, window_size):
    columns = list(y)
    data = []

    for i in range(window_size, y.shape[0], window_size):
        values = y.iloc[i].values
        zipped = zip(columns, values)
        a_dictionary = dict(zipped)
        data.append(a_dictionary)

    
    y_3d = y.loc[-199]
    y_3d.append(data, True)
    return y_3d

#### Variables

In [ ]:
dir_path = ''

event_id = {'1536': 1, '1537': 2, '1538': 3, '1539': 4, '1540': 5, '1541': 6, '1542': 7}

event_dictionary = {'elow/flexion': 1, 'elbow/extension': 2, 'arm/supination': 3,
              'arm/pronation': 4, 'hand/close': 5, 'hand/open': 6, 'rest': 7}
tmin = -0.75
tmax = 2

l_freq = 50
h_freq = 0.5

sfreq=256

n_samples = 42
time_steps=50
window_size = 704
n_channels = 61
n_classes = 7

n_epochs = 10

#### LSTM model

In [ ]:
model = keras.models.Sequential()
model.add(LSTM(units=128, input_shape=(window_size, n_channels), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=32, activation='relu'))
model.add(Dense(7, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=0.0001), metrics=['accuracy'])
model.summary()

In [ ]:
model_history = {}
for dir_name in tqdm(os.listdir(dir_path)[:1]):
    for file_name in tqdm(os.listdir(os.path.join(dir_path, dir_name))[1:2]):
        data_path = os.path.join(dir_path, dir_name, file_name)
        data = read_gdf(data_path)
        epochs = create_epochs(data=data, event_id=event_id, event_dictionary=event_dictionary, tmin=tmin, tmax=tmax)
        filtered_epochs = filtering_epochs(epochs, l_freq=l_freq, h_freq=h_freq)
        downsampled_epochs = downsampling_epochs(filtered_epochs, sfreq=sfreq)
        dataframed_epochs = epochs_to_dataframe(downsampled_epochs)
        X, y = df_to_x_y(dataframed_epochs)
        X = x_to_3d(X)
        y = y_to_3d(y, window_size=window_size)
        model_history[file_name] = model.fit(X, y, batch_size=1, shuffle=True, epochs=n_epochs)
    #if n_epochs > 4:
        #n_epochs -= 2

In [ ]:
history_loss = []
history_accuracy = []
for key, value in model_history.items():
    history_loss.append(value.history['loss'])
    history_accuracy.append(value.history['accuracy'])
    
    
plt.plot(history_loss, label='Loss')
plt.title('Loss for training data')
plt.ylabel('Loss')
plt.xlabel('No. epoch')
plt.legend(loc="upper left")
plt.show()

plt.plot(history_accuracy, label='Accuracy')
plt.title('Accuracy for training data')
plt.ylabel('Accuracy')
plt.xlabel('No. epoch')
plt.legend(loc='upper left')
plt.show()

#### Model evaluation

In [ ]:
results = {}
predictions = {}
for dir_name in tqdm(os.listdir(dir_path)[-14:-13]):
    for file_name in os.listdir(os.path.join(dir_path, dir_name)):
        data_path = os.path.join(dir_path, dir_name, file_name)
        data = read_gdf(data_path)
        epochs = create_epochs(data=data, event_id=event_id, event_dictionary=event_dictionary, tmin=tmin, tmax=tmax)
        filtered_epochs = filtering_epochs(epochs, l_freq=l_freq, h_freq=h_freq)
        downsampled_epochs = downsampling_epochs(filtered_epochs, sfreq=sfreq)
        dataframed_epochs = epochs_to_dataframe(downsampled_epochs)
        X, y = df_to_x_y(dataframed_epochs)
        X = x_to_3d(X)
        y = y_to_3d(y, window_size=window_size)
        results[file_name] = model.evaluate(X, y, batch_size=1)
        predictions[file_name] = model.predict(X, batch_size=1)

In [ ]:
model.save('EEG_10epochs_per_file')